In [61]:
import numpy as np
import itertools

X, Y = [], []

"""[2,3,84,5,1]
   [54,2,45,34,78]
   [23,34,45,76,88]"""

ts = [2,3,84,5,1,54,2,45,34,78,23,34,45,76,88]
values = np.array(ts)
values = values.astype('float32')
lag = 3

if len(ts) - lag <= 0:
    X.append(ts)
else:
    for i in range(len(ts) - lag):
        Y.append(ts[i + lag])
        # Substacted 96 for not knowing the day before
        ab = list(itertools.chain([ts[i+lag]], [ts[i+lag-1]]))
        X.append(ab)
        #X.append(ts[(i + lag)])
print(X)     
#X = list(X)
X, Y = np.array(X), np.array(Y)
print(X.shape)
# Reshaping the X array to an LSTM input shape 
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
X.shape

[[5, 84], [1, 5], [54, 1], [2, 54], [45, 2], [34, 45], [78, 34], [23, 78], [34, 23], [45, 34], [76, 45], [88, 76]]
(12, 2)


(12, 1, 2)

In [45]:
X.shape

(12, 3, 1)

In [70]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels
    
    WindowGenerator.split_window = split_window
    
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
          data=data,
          targets=None,
          sequence_length=self.total_window_size,
          sequence_stride=1,
          shuffle=True,
          batch_size=32,)

        ds = ds.map(self.split_window)

        return ds

    WindowGenerator.make_dataset = make_dataset


In [68]:
def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
        labels = tf.stack(
            [labels[:, :, self.column_indices[name]] for name in self.label_columns],
            axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

WindowGenerator.split_window = split_window

In [69]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)


TypeError: __init__() missing 3 required positional arguments: 'train_df', 'val_df', and 'test_df'

In [19]:
df_cons.isna().sum()

Datetime     0
Valeur      20
dtype: int64

In [20]:
df_temp.isna().sum()

Datetime             0
Temperatures (°C)    0
dtype: int64

In [16]:
import pandas as pd
df_cons = pd.read_csv('./Consumption_15min.csv')
df_temp = pd.read_csv('./Temp_15min.csv')

In [9]:
def predict_n_ahead(self, n_ahead: int):
    """
    A method to predict n time steps ahead
    """    
    X, _, _, _, _, _ = self.create_data_for_NN(use_last_n=self.lag2)   
    print(X.shape)

    # Making the prediction list 
    yhat = []

    for _ in range(n_ahead):
        # Making the prediction
        fc = self.model.predict(X)
        print('fc is')
        print(fc.shape)
        yhat.append(fc)

        for i in range(len(ts) - lag):
            Y.append(ts[i + lag])
            # Substacted 96 for not knowing the day before
            ab = list(itertools.chain([ts[i+lag - lag]], [ts[i+lag - lag2]], [holiday[i + lag]], [hour_cos[i + lag]], [hour_sin[i + lag]], [week_cos[i + lag]], [week_sin[i + lag]], [month_cos[i + lag]], [month_sin[i + lag]], [minute_cos[i + lag]], [minute_sin[i + lag]]))
            X.append(ab)

        # Creating a new input matrix for forecasting
        X = np.append(X, fc)

        # Ommiting the first variable
        X = np.delete(X, range(self.lag2))

        # Reshaping for the next iteration
        X = np.reshape(X, (X.shape[0], 1, 11))
        #(X.shape[0], 1, X.shape[1])
        print(X.shape)
    yhat = [y[0][0] for y in yhat]
    return yhat 

,Datetime,Valeur
0,2014-12-06 00:00:00,7.14
1,2014-12-06 00:15:00,7.56
2,2014-12-06 00:30:00,6.72
3,2014-12-06 00:45:00,6.93
4,2014-12-06 01:00:00,7.14
...,...,...
177787,2019-12-31 22:45:00,3.78
177788,2019-12-31 23:00:00,3.99
177789,2019-12-31 23:15:00,3.78
177790,2019-12-31 23:30:00,3.78


In [89]:
data1 = pd.read_pickle('Building-1-Roulart Media Group/building_1_cnsumptions.pkl')
data1.index = pd.to_datetime(data1.index)
#data1['Datetime'] = data1.index
data1

,Valeur
Date,
2014-01-06 02:00:00,6.51
2014-01-06 02:15:00,6.51
2014-01-06 02:30:00,6.72
2014-01-06 02:45:00,6.51
2014-01-06 03:00:00,6.93
...,...
2020-06-30 23:00:00,5.04
2020-06-30 23:15:00,3.36
2020-06-30 23:30:00,3.36


In [90]:
data1 = data1.sort_index()
data1 = data1.groupby(data1.index)['Valeur'].mean()
data1 = data1.loc[:'2020']


In [42]:
print(X)

[[ 2  3 84]
 [ 3 84  5]
 [84  5  1]
 [ 5  1 54]
 [ 1 54  2]
 [54  2 45]
 [ 2 45 34]]
